In [1]:
import time, os, json
from gridappsd import GridAPPSD,  topics as t

os.environ['GRIDAPPSD_USER'] = 'app_user'
os.environ['GRIDAPPSD_PASSWORD'] = '1234App'

gapps = GridAPPSD()
assert gapps.connected

In [2]:
topic = "goss.gridappsd.request.data.topology"

message = {
   "requestType": "GET_SWITCH_AREAS",
   "model_id":  '_C1C3E687-6FFD-C753-582B-632A27E28507',
   "resultFormat": "JSON"
}

dist_api_message = gapps.get_response(topic, message, timeout=30)

In [3]:
with open('dist_api_message.json', 'w') as f:
    json.dump(dist_api_message, f)

In [4]:
topic = "goss.gridappsd.request.data.topology"

message = {
   "requestType": "GET_BASE_TOPOLOGY",
   "model_id": "_EE71F6C9-56F0-4167-A14E-7F4C71F10EAA",
   "resultFormat": "JSON"
}

base_topo = gapps.get_response(topic, message, timeout=30)

In [5]:
topic = "goss.gridappsd.request.data.topology"

message = {
   "requestType": "GET_SNAPSHOT_TOPOLOGY",
   "model_id": "_EE71F6C9-56F0-4167-A14E-7F4C71F10EAA",
   "simulation_id": "670574834",
   "timestamp": "1644002615",
   "resultFormat": "JSON"
}

service_message = gapps.get_response(topic, message, timeout=90)

KeyboardInterrupt: 

In [ ]:
from gridappsd import topics as t
topic = t.REQUEST_POWERGRID_DATA

message = {
    "requestType": "QUERY_MODEL_NAMES",
    "resultFormat": "JSON"
}

gapps.get_response(topic, message)

In [7]:
from topologyprocessor import TopologyProcessor
from topologydictionary import TopologyDictionary
from networkmodel import NetworkModel
from distributedtopology import DistributedTopology

In [8]:
model_mrid = "_EE71F6C9-56F0-4167-A14E-7F4C71F10EAA" #final9500nodeb

In [9]:
# Base Topology, direct invocation
Topology = TopologyProcessor()
NetworkTopo = Topology.get_base_topology(model_mrid)

In [11]:
# Base Topology, use library methods
Topology = TopologyDictionary(gapps, model_mrid)
network = NetworkModel(gapps)
network.build_equip_dicts(model_mrid, Topology)
EqTypes = ['ACLineSegment', 'PowerTransformer', 'TransformerTank', 'SynchronousMachine']
Topology.build_linknet(EqTypes)
Topology.EquipDict['LoadBreakSwitch']['_494810CC-1A00-4EE8-8360-484FA7C19D01']['open'] = 1
Topology.update_switches()
Topology.build_feeder_islands()

In [12]:
# missing nodes, to be fixed in 2022.02 release
nodes = list(NetworkTopo.ConnNodeDict.keys())

for i1 in range(len(nodes)):
    if not Topology.ConnNodeDict[nodes[i1]]['Feeder']:
        print(Topology.ConnNodeDict[nodes[i1]]['name'], nodes[i1])

hvmv115_hsb1 _A74C1A0C-2A1C-4418-AA45-715A709B27EC
hvmv115_hsb2 _C2BF3DC0-C44D-4C33-968C-8863A63E9C38
hvmv69s1s2_1 _345B4E95-D766-427B-BB93-C6F7B323F382
hvmv69s1s2_10 _8BA4F660-4379-4E8A-A25E-6A1650D59D68
hvmv69s1s2_2 _E5DA1E1B-4C63-443A-8D0B-076D0DF3E971
hvmv69s1s2_3 _B136057A-B130-402D-AC66-4F42935A8971
hvmv69s1s2_4 _D0E02351-9D63-4808-B405-1693749D2D57
hvmv69s1s2_5 _857A65D2-2993-4985-AF79-E811BD9C5898
hvmv69s1s2_6 _B6529F2B-373E-4552-A7FD-0E87187CE28C
hvmv69s1s2_7 _35BD37CD-8F75-4220-BA6C-1B4AB5FA0E09
hvmv69s1s2_8 _E0FD90E9-8057-4790-AECA-330D63FE9B98
hvmv69s1s2_9 _73AAF2C9-FBE4-4C8E-A3F9-D78F1269A2FF
hvmv69s2s3_1 _DE827511-2D96-4C9A-BECC-FCCFF246434B
hvmv69s2s3_2 _B993AAEC-CAE9-450F-B547-55272C9A946D
hvmv69sub1_lsb2 _66E60B0C-5E62-432E-9FD4-98248753D291
hvmv69sub2_lnb _B0426C55-90EB-486D-84FD-3510ED53564C
m1026chp-2 _E0D6F386-AFB8-4037-BEC9-96963A9E866C
m2001-ess1 _E118EFA9-CF00-4DC5-9D6C-5D7486C0F7E5
m2001-ess2 _B3C43D4D-489A-4A90-B56F-D1956053FC8B
regxfmr_hvmv69sub1_lsb1 _D10962

In [ ]:
Topology.Islands

In [ ]:
Topology.Feeders['feeder_0']

In [ ]:
import topo_meas_queries as queries
model_mrid = "C1C3E687-6FFD-C753-582B-632A27E28507"
TapChangerQuery = queries.get_all_tapchangers(gapps, model_mrid)
for i0 in range(len(TapChangerQuery)):
    print(i0)

In [ ]:
message = {
   "requestType": "GET_BASE_Topology",
   "modelID": "_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62",
   "resultFormat": "JSON"
}

topic = "goss.gridappsd.request.data.topology"

gapps.get_response(topic, message, timeout=30)

In [ ]:
DistTopo = DistributedTopology(gapps, model_mrid)
message = DistTopo.create_switch_areas(model_mrid)

In [ ]:
root = ''
if root: print('rrot')

In [ ]:
from gridappsd import topics as t
topic = t.REQUEST_POWERGRID_DATA

message = {
    "requestType": "QUERY_OBJECT_DICT",
    "modelId": '_C1C3E687-6FFD-C753-582B-632A27E28507',
    "objectId": "_9DA10945-2EDC-43C9-A748-B079D826B0DB",
    "resultFormat": "CSV"
}

gapps.get_response(topic, message)